In [1]:
!pip install transformers
!pip install sentencepiece

In [2]:
import torch
import pandas as pd
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import numpy as np
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
#GPU 사용
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [4]:
train_data = pd.read_csv("/content/drive/MyDrive/자연어처리음성인식/조상연/BERT 감정분류/train_data.csv")
valid_data = pd.read_csv("/content/drive/MyDrive/자연어처리음성인식/조상연/BERT 감정분류/valid_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/자연어처리음성인식/조상연/BERT 감정분류/test_data.csv")

In [5]:
classes = train_data['label'].unique().tolist()
classes

['기쁨', '불안', '슬픔', '분노']

In [6]:
for i in range(len(train_data)):
    idx = classes.index(train_data.iloc[i][0])
    train_data.iloc[i][0] = idx

In [7]:
for i in range(len(valid_data)):
    idx = classes.index(valid_data.iloc[i][0])
    valid_data.iloc[i][0] = idx

for i in range(len(test_data)):
    idx = classes.index(test_data.iloc[i][0])
    test_data.iloc[i][0] = idx

In [8]:
class BERTDataset(Dataset):
  def __init__(self, data, max_len):
    super(BERTDataset, self).__init__()
    self.data = data
    self.max_len = max_len
    self.tokenizer = AutoTokenizer.from_pretrained("klue/bert-base", use_fast = True)

    self.inputs = [self.convert_token([data.iloc[i][1]]) for i in range(len(self.data))]
    self.label = [np.int32(data.iloc[i]['label']) for i in range(len(self.data))]

  def convert_token(self, data):
    token = self.tokenizer.encode(data[0])
    attention_mask = [1] * len(token) + [0] * (self.max_len - len(token))
    token = token + self.tokenizer.convert_tokens_to_ids(["[PAD]"] * (self.max_len - len(token)))
    return [np.int32(attention_mask), np.int32(token)]

  def __getitem__(self, idx):
    return self.inputs[idx][0], self.inputs[idx][1], self.label[idx]
  
  def __len__(self):
    return len(self.label)

In [9]:
train_dataset = BERTDataset(train_data, 128)

In [10]:
valid_dataset = BERTDataset(valid_data, 128)
test_dataset = BERTDataset(test_data, 128)

In [11]:
class EmotionClassifier(nn.Module):
  def __init__(self, num_classes = 4):
    super(EmotionClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("klue/bert-base")
    self.bert.requires_grad = True
    self.classifier = nn.Linear(768, num_classes)

  def forward(self, input_ids, attention_mask):
    x = self.bert(input_ids, attention_mask).pooler_output
    x = self.classifier(x)
    return x

In [12]:
model = EmotionClassifier().cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [14]:
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 256
learning_rate =  5e-5

In [15]:
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [16]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

In [17]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [19]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [21]:
best_acc = 0
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (attention_mask, token_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.long().to(device)
        out = model(token_ids, attention_mask)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("EPOCH {} [{}/{}]  >>>  loss : {:.6f}\t  train_acc : {:.3f}".format(e+1, batch_id+1,len(train_dataloader),
                                                                           loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("EPOCH {}  >>>  loss : {:.6f}\t  train_acc : {:.3f}".format(e+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (attention_mask, token_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.long().to(device)
        out = model(token_ids, attention_mask)
        test_acc += calc_accuracy(out, label)
    print("EPOCH {}  >>>  test_acc : {:.3f}".format(e+1, test_acc / (batch_id+1)))
    if best_acc < test_acc:
        torch.save(model.state_dict(), '/content/drive/MyDrive/자연어처리음성인식/조상연/BERT 감정분류/KoBert_base_klue.pt')
        best_acc = test_acc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2158 [00:00<?, ?it/s]

EPOCH 1 [1/2158]  >>>  loss : 1.451906	  train_acc : 0.188
EPOCH 1 [257/2158]  >>>  loss : 1.036505	  train_acc : 0.356
EPOCH 1 [513/2158]  >>>  loss : 0.652018	  train_acc : 0.498
EPOCH 1 [769/2158]  >>>  loss : 0.829219	  train_acc : 0.572
EPOCH 1 [1025/2158]  >>>  loss : 0.460903	  train_acc : 0.614
EPOCH 1 [1281/2158]  >>>  loss : 0.704930	  train_acc : 0.642
EPOCH 1 [1537/2158]  >>>  loss : 0.681814	  train_acc : 0.658
EPOCH 1 [1793/2158]  >>>  loss : 0.727212	  train_acc : 0.672
EPOCH 1 [2049/2158]  >>>  loss : 0.958371	  train_acc : 0.682
EPOCH 1  >>>  loss : 0.511582	  train_acc : 0.686


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/263 [00:00<?, ?it/s]

...

EPOCH 19  >>>  test_acc : 0.751


  0%|          | 0/2158 [00:00<?, ?it/s]

EPOCH 20 [1/2158]  >>>  loss : 0.000068	  train_acc : 1.000
EPOCH 20 [257/2158]  >>>  loss : 0.000043	  train_acc : 0.996
EPOCH 20 [513/2158]  >>>  loss : 0.000059	  train_acc : 0.995
EPOCH 20 [769/2158]  >>>  loss : 0.000048	  train_acc : 0.996
EPOCH 20 [1025/2158]  >>>  loss : 0.000050	  train_acc : 0.996
EPOCH 20 [1281/2158]  >>>  loss : 0.000045	  train_acc : 0.996
EPOCH 20 [1537/2158]  >>>  loss : 0.000118	  train_acc : 0.996
EPOCH 20 [1793/2158]  >>>  loss : 0.596808	  train_acc : 0.996
EPOCH 20 [2049/2158]  >>>  loss : 0.000068	  train_acc : 0.996
EPOCH 20  >>>  loss : 0.000041	  train_acc : 0.996


  0%|          | 0/263 [00:00<?, ?it/s]

EPOCH 20  >>>  test_acc : 0.751


In [22]:
model = EmotionClassifier().cuda()
model.load_state_dict(torch.load('/content/drive/MyDrive/자연어처리음성인식/조상연/BERT 감정분류/KoBert_base_klue.pt'))
model.eval()
st = time.time()
test_acc = 0
for batch_id, (attention_mask, token_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    attention_mask = attention_mask.long().to(device)
    label = label.long().to(device)
    out = model(token_ids, attention_mask)
    max_vals, max_indices = torch.max(out, 1)
    test_acc += (max_indices == label).sum().data.cpu().numpy()
ed = time
print("TEST  >>>  test_acc : {:.3f}".format(test_acc / len(test_dataloader.dataset)))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm=

  0%|          | 0/7 [00:00<?, ?it/s]

TEST  >>>  test_acc : 0.790
